# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 13, 872, 8, 106, 14, 31, 88, 45, 77, 2305, 18, 735, 14, 291, 13, 482, 17, 111, 1588, 133, 13, 66, 473, 8, 40, 14, 20, 21, 1054, 56, 1100, 5, 685, 86, 136, 16, 608, 21, 4, 226, 360, 7, 4, 20, 11, 2, 957, 370, 2, 9, 66, 616, 10, 10, 13, 482, 4, 293, 282, 18, 231, 141, 6, 20, 5, 2, 12, 18, 735, 9, 14, 10, 10, 298, 1248, 1692, 84, 37, 166, 278, 23, 325, 2, 359, 35, 1338, 42, 2, 7, 325, 11, 4933, 34, 2, 56, 142, 432, 7, 2, 10, 10, 138, 134, 3818, 2048, 7, 167, 5, 4, 293, 109, 8, 2, 4, 311, 19, 35, 326, 7, 1715, 325, 40, 6, 1392, 449, 591, 216, 23, 51, 6, 2696, 10, 10, 106, 14, 8, 79, 35, 326, 7, 89, 364, 4, 20, 1809, 70, 809]
--- Label ---
0


Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1646592/1641221 [==============================] - 0s 0us/step
--- Review (with words) ---
['the', 'was', 'subject', 'in', 'character', 'as', 'by', 'most', 'if', 'will', 'claim', 'but', 'needs', 'as', 'someone', 'was', 'lead', 'movie', 'plot', 'cliché', 'scene', 'was', 'had', 'children', 'in', 'just', 'as', 'on', 'not', 'filmmakers', 'she', 'spirit', 'to', 'due', 'how', 'scenes', 'with', 'career', 'not', 'of', 'script', 'hollywood', 'br', 'of', 'on', 'this', 'and', 'previous', 'awful', 'and', 'it', 'had', 'sad', 'i', 'i', 'was', 'lead', 'of', 'watched', 'everything', 'but', 'minutes', 'should', 'is', 'on', 'to', 'and', 'that', 'but', 'needs', 'it', 'as', 'i', 'i', 'said', 'aspect', 'began', 'great', 'like', 'find', 'sense', 'are', 'black', 'and', 'kids', 'so', 'issues', "it's", 'and', 'br', 'black', 'this', 'marketing', 'who', 'and', 'she', 'back', 'went', 'br', 'and', 'i', 'i', 'such', 'while', 'ho', 'learned', 'br', 'going', 'to', 'of', 'watched', 'being', 'in', 'and', 'of', 'night',

In [4]:
type(X_train)

numpy.ndarray

In [9]:
len(X_train[2])

274

In [10]:
len(max(X_train, key=len))

2494

In [12]:
len(min(X_train, key=len))

11

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:  

Maximum Review length is 2494 & minimum is 11.

...


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [13]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test

X_train_pad = sequence.pad_sequences(X_train, maxlen=500)
X_test_pad  = sequence.pad_sequences(X_test, maxlen=500)

In [14]:
X_train_pad.shape

(25000, 500)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [15]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
Embedding_size = 64
model = Sequential()
model.add(Embedding(vocabulary_size, Embedding_size, input_length = max_words))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 64)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 386,101
Trainable params: 386,101
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

...

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [17]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [18]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 5

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
# TODO: Train your model
model.fit(X_train_pad, y_train, validation_split = 0.01, batch_size = batch_size, epochs = num_epochs)

Train on 24750 samples, validate on 250 samples
Epoch 1/5
24750/24750 [==============================] - 614s 25ms/step - loss: 0.4761 - acc: 0.7623 - val_loss: 0.2700 - val_acc: 0.9040
Epoch 2/5
24750/24750 [==============================] - 613s 25ms/step - loss: 0.3084 - acc: 0.8742 - val_loss: 0.2584 - val_acc: 0.8960
Epoch 3/5
24750/24750 [==============================] - 617s 25ms/step - loss: 0.2495 - acc: 0.9030 - val_loss: 0.2469 - val_acc: 0.9040
Epoch 4/5
24750/24750 [==============================] - 625s 25ms/step - loss: 0.2095 - acc: 0.9194 - val_loss: 0.2337 - val_acc: 0.9040
Epoch 5/5
24750/24750 [==============================] - 613s 25ms/step - loss: 0.1808 - acc: 0.9297 - val_loss: 0.2578 - val_acc: 0.8840


In [23]:
import os
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join('Untitled Folder', model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [25]:
# Evaluate your model on the test set
scores = model.evaluate(X_test_pad, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.86848


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

...

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!